# State Design Pattern

In [1]:
class IState{
    public:
    virtual ~IState() {}
    virtual void insertQuarter() = 0;
    virtual void ejectQuarter() = 0;
    virtual void turnCrank() = 0;
    virtual void dispense() = 0;
};

In [2]:
class CSoldState: public IState
{
public:
CSoldState(){}

virtual void insertQuarter() override final {}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}
    
};

In [3]:
class CSoldOutState: public IState
{
public:
CSoldOutState(){}
virtual ~CSoldOutState() override {}
virtual void insertQuarter() override final {}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}


    
};

In [4]:
#include <iostream>
class CGumballMachine;

class CNoQuarterState: public IState
{
public:
CNoQuarterState(CGumballMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CNoQuarterState() override {}
virtual void insertQuarter() override final 
{
    std::cout << "You inserted a quarter\n"
    m_GumballMachine.setState();
}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}
    
private:
CGumballMachine& m_GumballMachine
    
};

In [5]:
class CHasQuarterState: public IState
{
public:
CHasQuarterState(){}
virtual ~CHasQuarterState() override {}
virtual void insertQuarter() override final {}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}
    
};

In [6]:
class CWinnerState: public IState
{
public:
CWinnerState(){}
virtual ~CWinnerState() override {}
virtual void insertQuarter() override final {}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}
    
};

In [7]:
#include <iostream>
class CGumballMachine
{
    public:
    CGumballMachine(int numberGumballs):m_state(nullptr),m_count(numberGumballs)
    {
        soldOutState = new CSoldOutState;
        noQuarterState = new CNoQuarterState;
        hasQuarterState = new CHasQuarterState;
        soldState = new CSoldState;
        m_state = soldOutState;
        if (m_count > 0)
        {
            m_state = noQuarterState;
        }
    }
    virtual ~CGumballMachine(){}
    
    void insertQuarter()
    {
        m_state->insertQuarter();
    }
    
    void ejectQuarter()
    {
        m_state->ejectQuarter();
    }
    
    void turnCrank()
    {
        m_state->turnCrank();
        m_state->dispense();
    }
    
    void setState(IState* state)
    {
        m_state = state;
    }
    
    void releaseBall()
    {
        std::cout << "A gumball comes rolling out the slot...";
        if (m_count !=0)
        {
            m_count = m_count - 1;
        }
    }
    
    private:
    IState* soldOutState;
    IState* noQuarterState;
    IState* hasQuarterState;
    IState* soldState;
    
    IState* m_state;
    int m_count = 0;
}